In [2]:
import requests
import json
import pandas as pd
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/south%20sudan?unitGroup=metric&key=U2TSBQHYKRYCZGKXDML36YX38&contentType=json'
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)
df.head()

ValueError: All arrays must be of the same length

In [7]:
print(data['days'][0].keys())

dict_keys(['datetime', 'datetimeEpoch', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise', 'sunriseEpoch', 'sunset', 'sunsetEpoch', 'moonphase', 'conditions', 'description', 'icon', 'stations', 'source', 'hours'])


In [16]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations

df_art = pd.read_csv("data/food_crises_cleaned.csv")
df_regions = df_art[['district_code','centx', 'centy']].drop_duplicates()


start = datetime(2007, 1, 1)
end = datetime(2020, 2, 28)
location = Point(lat=df_regions['centy'][0], lon=df_regions['centx'][0], alt=70)
data = Daily(location, start, end)
data = data.fetch()

In [17]:
from tqdm import tqdm
tqdm.pandas()
def get_closest_weather_station(region):
    stations = Stations()
    stations = stations.nearby(region[0], region[1])
    station = stations.fetch(1)
    station = station.sort_values(by='distance')
    name = station[['name']].values[0][0]
    
    return station[['latitude', 'longitude', 'elevation']].values[0].tolist()



df_regions['closest_station'] = df_regions[['centx', 'centy']].progress_apply(get_closest_weather_station, axis=1)

100%|██████████| 78/78 [00:01<00:00, 57.24it/s]


In [18]:
df_regions.head()

,district_code,centx,centy,closest_station
0,74,32.00486,6.465644,"[31.6667, 6.15, 142.0]"
158,75,28.02439,7.592506,"[26.5, 8.4167, 558.0]"
316,76,26.72969,9.334756,"[26.5, 8.4167, 558.0]"
474,77,30.83354,3.737241,"[30.5667, 2.8667, 397.0]"
632,78,32.21030,3.910535,"[32.3833, 3.8167, 450.0]"


In [15]:
df_art

,country,district_code,district,centx,centy,year_month,year,month,ipc,ha,...,et_anom,count_violence,sum_fatalities,food_price_idx,area,cropland_pct,pop,ruggedness_mean,pasture_pct,date
0,South Sudan,74,Bor,32.00486,6.465644,2007_01,2007,1,NaN,NaN,...,0.048880,0,0,0.561363,14008.3300,7.961984,239344.0,11393.760,90.8503,2007-01-01
1,South Sudan,74,Bor,32.00486,6.465644,2007_02,2007,2,NaN,NaN,...,-0.457928,0,0,0.565693,14008.3300,7.961984,239344.0,11393.760,90.8503,2007-02-01
2,South Sudan,74,Bor,32.00486,6.465644,2007_03,2007,3,NaN,NaN,...,-0.590175,0,0,0.592437,14008.3300,7.961984,239344.0,11393.760,90.8503,2007-03-01
3,South Sudan,74,Bor,32.00486,6.465644,2007_04,2007,4,NaN,NaN,...,-1.845536,0,0,0.605352,14008.3300,7.961984,239344.0,11393.760,90.8503,2007-04-01
4,South Sudan,74,Bor,32.00486,6.465644,2007_05,2007,5,NaN,NaN,...,-2.972735,0,0,0.618699,14008.3300,7.961984,239344.0,11393.760,90.8503,2007-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12319,South Sudan,151,Malakal,31.64280,9.658457,2019_10,2019,10,3.0,0.0,...,6.053181,1,0,82.872570,757.7855,13.068690,102228.0,7145.909,85.5000,2019-10-01
12320,South Sudan,151,Malakal,31.64280,9.658457,2019_11,2019,11,NaN,NaN,...,8.745254,0,0,87.728720,757.7855,13.068690,102228.0,7145.909,85.5000,2019-11-01
12321,South Sudan,151,Malakal,31.64280,9.658457,2019_12,2019,12,NaN,NaN,...,1.698060,0,0,93.566750,757.7855,13.068690,102228.0,7145.909,85.5000,2019-12-01
12322,South Sudan,151,Malakal,31.64280,9.658457,2020_01,2020,1,NaN,NaN,...,1.046844,0,0,106.793600,757.7855,13.068690,163171.3,7145.909,85.5000,2020-01-01


In [23]:
start = datetime(2007, 1, 1)
end = datetime(2020, 2, 28)


for index, row in df_regions.iterrows():
    data = Daily(Point(*row['closest_station']), start, end).fetch()
    data.to_csv(f'data/weather_monthly/{row["district_code"]}.csv')


In [21]:
data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,


In [4]:
data.index = pd.to_datetime(data.index, format='%Y-%m-%d')
data_grouped = data.groupby(by=[data.index.year, data.index.month]).mean()



dates = pd.to_datetime(data_grouped.index, format='(%Y, %m)')
data_grouped.index = dates
data_grouped.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
2007-01-01,12.280645,5.030769,20.193103,0.000000,NaN,61.500000,6.059091,NaN,NaN,NaN
2007-02-01,15.353571,8.629630,22.192308,0.088462,NaN,126.000000,13.300000,NaN,NaN,NaN
2007-03-01,17.022581,9.284615,24.116000,0.034483,NaN,172.857143,20.640000,NaN,NaN,NaN
2007-04-01,22.550000,15.785714,29.566667,1.616667,NaN,55.666667,19.334783,NaN,NaN,NaN
2007-05-01,28.212903,20.420690,35.220690,0.016129,NaN,236.500000,16.600000,NaN,NaN,NaN


In [ ]:
import requests
import sys

import json
                

response = requests.request("GET", "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/south%20sudan/monthtodate?unitGroup=metric&key=U2TSBQHYKRYCZGKXDML36YX38&contentType=json")
if response.status_code!=200:
  print('Unexpected Status code: ', response.status_code)
  sys.exit()  


# Parse the results as JSON
jsonData = response.json()
jsonData

{'queryCost': 457,
 'latitude': 4.83975,
 'longitude': 31.5976,
 'resolvedAddress': 'South Sudan',
 'address': 'south sudan',
 'timezone': 'Africa/Juba',
 'tzoffset': 2.0,
 'description': 'Similar temperatures continuing with no rain expected.',
 'days': [{'datetime': '2023-09-01',
   'datetimeEpoch': 1693519200,
   'tempmax': 36.0,
   'tempmin': 19.5,
   'temp': 26.5,
   'feelslikemax': 36.1,
   'feelslikemin': 19.5,
   'feelslike': 26.8,
   'dew': 17.8,
   'humidity': 63.7,
   'precip': 1.3,
   'precipprob': 100.0,
   'precipcover': 20.83,
   'preciptype': ['rain'],
   'snow': 0.0,
   'snowdepth': 0.0,
   'windgust': 14.4,
   'windspeed': 11.9,
   'winddir': 62.4,
   'pressure': 1010.9,
   'cloudcover': 18.5,
   'visibility': 22.5,
   'solarradiation': 304.1,
   'solarenergy': 26.5,
   'uvindex': 10.0,
   'severerisk': 30.0,
   'sunrise': '05:47:32',
   'sunriseEpoch': 1693540052,
   'sunset': '17:59:58',
   'sunsetEpoch': 1693583998,
   'moonphase': 0.54,
   'conditions': 'Rain',
  

In [59]:
df_regions.values[0][2]

[31.6667, 6.15, 142.0]

In [43]:
df_regions['closest_station'][0]

'60581'

In [45]:
stations = Stations()


In [27]:
stations = Stations()
stations = stations.nearby(df_regions['centy'][0], df_regions['centx'][0], 200000)

In [28]:
stations = stations.fetch()

In [29]:
stations.sort_values(by='distance')

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
62941,Juba,SD,BG,62941,HSSJ,4.8667,31.6000,460.0,Africa/Khartoum,1949-03-01,2023-08-26,1957-07-01,2023-09-11,1949-01-01,2011-01-01,183351.123611
HSAK0,Akobo,SD,ANL,<NA>,HSAK,7.7833,33.0167,402.0,Africa/Khartoum,2004-02-28,2012-12-18,NaT,NaT,NaT,NaT,184202.775493


In [7]:
data.plot(y=['tavg', 'tmin', 'tmax'])
plt.show()

TypeError: no numeric data to plot

In [9]:
vars(location)

{'_lat': 6.465644,
 '_lon': 32.00486,
 '_alt': 70,
 '_stations': Index([], dtype='string', name='id')}

In [33]:
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm
import pandas as pd
tqdm.pandas()
sentiment_analysis = pipeline("text-classification", truncation=True)
df = pd.read_csv("data/all_africa_southsudan.csv")



def analyze_sentiment(text):
    return sentiment_analysis(text)



df['paragraphs'] = df['paragraphs'].astype(str)

df['sentiment'] = df['paragraphs'].progress_apply(analyze_sentiment)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
100%|██████████| 18226/18226 [2:25:02<00:00,  2.09it/s]  


In [34]:
df.to_csv("data/all_africa_southsudan_sentiment_default.csv")

In [8]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available()).__annotations__


Torch version: 2.0.1+cpu
Is CUDA enabled? False


In [3]:
df.loc[2, 'paragraphs']

"Former NBA star Luol Ajou Deng has been elected the new President of South Sudan Basketball Federation after winning 24 votes out of 25. In the weekend election organised by the South Sudan Basketball Federation, Mr Ajou sailed through unchallenged, with Shengety Dhieu, the head of the election committee, announcing his win. Mr Marier Anyuat was elected secretary general and Mr Deng Biar Deng the finance secretary. According to the federation's constitution, Mr Ajou has the mandate to serve for four years. Mr Ajou, born 16 April 1985 is a South Sudanese-British former professional basketball player, a two-time NBA All-Star and was named to the NBA All-Defensive Second Team in 2012. Mr Ajou's family fled the country when he was a child, eventually settling in the United Kingdom. He became a British citizen in 2006, and has played for the Great Britain national team. After playing college basketball for the Duke Blue Devils, Mr Ajou was selected by the Chicago Bulls in the first round o

In [55]:
def parse_sentiment(sentiment):
    if sentiment[0]['label'] == 'POSITIVE':
        return sentiment[0]['score']
    else:
        return -sentiment[0]['score']
    

df['sentiment'] = df['sentiment'].progress_apply(parse_sentiment)

100%|██████████| 18520/18520 [00:00<00:00, 1027996.64it/s]


In [57]:
df.to_csv("data/articles_summary_cleaned.csv", index=False)

In [52]:
analyze_sentiment(df['summary'][2500])

[{'label': 'POSITIVE', 'score': 0.9931449890136719}]

In [37]:
df.head()

,title,date,paragraphs,publisher
0,South Sudan: A Former Child Soldier Fights to ...,2021-02-12,"Sara, who was abducted at age 12, spent two ye...",Thomson Reuters Foundation (London)
1,South Sudan: #budget2019 - Will It Lead to Mor...,2019-02-21,Legal claims against the department now amount...,Bhekisisa (Johannesburg)
2,South Sudan: Retired NBA Star Elected Basketba...,2019-11-25,Former NBA star Luol Ajou Deng has been electe...,The East African (Nairobi)
3,"South Sudan - Political Violence On the Rise, ...",2022-02-11,UN human rights experts have warned of increas...,UN News Service
4,South Sudan: S. Sudan Receives First Astrazene...,2021-03-25,"Juba — South Sudan on Thursday received 132,00...",The East African (Nairobi)


In [29]:
df['paragraphs'] = df['paragraphs'].astype(str)


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18226 entries, 0 to 18225
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       18226 non-null  object
 1   date        18226 non-null  object
 2   paragraphs  18226 non-null  object
 3   publisher   18226 non-null  object
dtypes: object(4)
memory usage: 569.7+ KB


In [22]:
import torch
import json
data = torch.load('train-batch.pt')
data.keys()

dict_keys(['static_categorical_features', 'static_real_features', 'past_time_features', 'past_values', 'past_observed_mask', 'future_time_features', 'future_values', 'future_observed_mask'])

In [47]:
from transformers import TimeSeriesTransformerModel, TimeSeriesTransformerConfig
configuration = TimeSeriesTransformerConfig(prediction_length=12)
model = TimeSeriesTransformerModel(configuration)
time_series = data
output = model(time_series, past_time_features='past_time_features', past_observed_mask='past_observed_mask')

TypeError: string indices must be integers

In [32]:
data['future_observed_mask'].size()

torch.Size([64, 24])

In [14]:
import pandas as pd

pd.DataFrame.from_dict(data, columns=data.keys(), orient='index')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [23]:
import pandas as pd

df = pd.read_csv("E:\Bestanden\data\BACI_HS12_V202301\BACI_HS12_Y2019_V202301.csv")
colums = {"t": "year",
          "k": "productcode",
          "i": "importer",
          "j": "exporter",
          "v": "value",
          "q": "quantity"}
df.rename(columns=colums, inplace=True)
# df_country_codes = pd.read_csv("E:\Bestanden\data\BACI_HS12_V202301\country_codes_V202301.csv") 
# df['importer'] = df['importer'].map(pd.Series(df_country_codes.country_name_abbreviation, index=df_country_codes.country_code))
# df['exporter'] = df['exporter'].map(pd.Series(df_country_codes.country_name_abbreviation, index=df_country_codes.country_code))
df.head()

,year,importer,exporter,productcode,value,quantity
0,2019,4,24,481940,1.603,1.000
1,2019,4,24,847141,1.114,0.022
2,2019,4,31,70310,6.275,22.003
3,2019,4,31,80211,2.434,0.205
4,2019,4,31,80420,0.463,0.061


In [35]:
df[(df["importer"] == 728) | (df["exporter"] == 728)]
df_imports = df[df["importer"] == 728]
df_exports = df[df["exporter"] == 728]
df_imports.value.max()

1533176.767

,country_code,country_name_abbreviation,country_name_full,iso_2digit_alpha,iso_3digit_alpha
200,728,South Sudan,South Sudan,SS,SSD


In [38]:
import requests

url = "https://comtrade.un.org/data/cache/classificationH0.json"
content = requests.get(url).json()

In [42]:
content['results']

[{'id': 'ALL', 'text': 'All HS1992 commodities', 'parent': '#'},
 {'id': 'TOTAL', 'text': 'Total of all HS1992 commodities', 'parent': '#'},
 {'id': 'AG2', 'text': 'AG2 - All 2-digit HS1992 commodities', 'parent': '#'},
 {'id': 'AG4', 'text': 'AG4 - All 4-digit HS1992 commodities', 'parent': '#'},
 {'id': 'AG6', 'text': 'AG6 - All 6-digit HS1992 commodities', 'parent': '#'},
 {'id': '01', 'text': '01 - Animals; live', 'parent': 'TOTAL'},
 {'id': '0101',
  'text': '0101 - Horses, asses, mules and hinnies; live',
  'parent': '01'},
 {'id': '010111',
  'text': '010111 - Horses; live, pure-bred breeding animals',
  'parent': '0101'},
 {'id': '010119',
  'text': '010119 - Horses; live, other than pure-bred breeding animals',
  'parent': '0101'},
 {'id': '010120',
  'text': '010120 - Asses, mules and hinnies; live',
  'parent': '0101'},
 {'id': '0102', 'text': '0102 - Bovine animals; live', 'parent': '01'},
 {'id': '010210',
  'text': '010210 - Bovine animals; live, pure-bred breeding animal

In [46]:
categories = [x for x in content['results'] if x["parent"] == "0303"]
categories

[{'id': '0101',
  'text': '0101 - Horses, asses, mules and hinnies; live',
  'parent': '01'},
 {'id': '0102', 'text': '0102 - Bovine animals; live', 'parent': '01'},
 {'id': '0103', 'text': '0103 - Swine; live', 'parent': '01'},
 {'id': '0104', 'text': '0104 - Sheep and goats; live', 'parent': '01'},
 {'id': '0105',
  'text': '0105 - Poultry; live, fowls of the species gallus domesticus, ducks, geese, turkeys and guinea fowls',
  'parent': '01'},
 {'id': '0106',
  'text': '0106 - Animals, n.e.s. in chapter 01; live',
  'parent': '01'}]

In [68]:
import re

categories = [x["text"].split("-")[1].strip() for x in content['results'] if re.search(r"\A[0-9]{4}\s", x["text"])]
categories

['Horses, asses, mules and hinnies; live',
 'Bovine animals; live',
 'Swine; live',
 'Sheep and goats; live',
 'Poultry; live, fowls of the species gallus domesticus, ducks, geese, turkeys and guinea fowls',
 'Animals, n.e.s. in chapter 01; live',
 'Meat of bovine animals; fresh or chilled',
 'Meat of bovine animals; frozen',
 'Meat of swine; fresh, chilled or frozen',
 'Meat of sheep or goats; fresh, chilled or frozen',
 'Meat of horses, asses, mules or hinnies; fresh, chilled or frozen',
 'Edible offal of bovine animals, swine, sheep, goats, horses, asses, mules or hinnies; fresh, chilled or frozen',
 'Meat and edible offal of poultry; of the poultry of heading no. 0105, (ie fowls of the species gallus domesticus), fresh, chilled or frozen',
 'Meat and edible meat offal, n.e.s. in chapter 2; fresh, chilled or frozen',
 'Pig fat, free of lean meat, and poultry fat, (not rendered), fresh, chilled, frozen, salted, in brine, dried or smoked',
 'Meat and edible meat offal; salted, in brin

In [59]:
content['results'][10]

{'id': '0102', 'text': '0102 - Bovine animals; live', 'parent': '01'}

In [72]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
topic_model = BERTopic(language="english", calculate_probabilities=True)
topics, probs = topic_model.fit_transform(categories)
topics_per_class = topic_model.topics_per_class(categories, classes=["Food items", "Non food items"])

ValueError: All arrays must be of the same length

In [75]:
from bertopic.representation import ZeroShotClassification
from bertopic import BERTopic


candidate_topics = ["Food items", "Non food items"]
representation_model = ZeroShotClassification(candidate_topics)

# Use the representation model in BERTopic on top of the default pipeline
topic_model = BERTopic(representation_model=representation_model)
topics, probs = topic_model.fit_transform(categories)

In [78]:
topic_model._extract_topics

<bound method BERTopic._extract_topics of <bertopic._bertopic.BERTopic object at 0x0000016B73DCBB80>>

In [ ]:
import pandas as pd

df_fc = pd.read_csv("data/food_crises_cleaned.csv") 

df_fc.value_counts

In [25]:
import pandas as pd
import ast
df = pd.read_csv("data/all_africa_southsudan_sentiment.csv")

df.sentiment = df.sentiment.apply(ast.literal_eval)
df.sentiment = df.sentiment.apply(lambda x: x[0])

In [27]:
df["sentiment"].loc[lambda x : x["label"] != "LABEL_0"]

KeyError: 'label'

In [31]:
df["label"] = df["sentiment"].apply(lambda x: x["label"])
df["label"].value_counts()

LABEL_0    18216
LABEL_1       10
Name: label, dtype: int64

In [32]:
df[df["label"] == "LABEL_1"]

,Unnamed: 0,title,date,paragraphs,publisher,sentiment,label
2034,2034,East Africa: Govt to Appoint Judges in EACJ Di...,2017-03-20,South Sudan will soon appoint a judge to sit i...,The East African (Nairobi),"{'label': 'LABEL_1', 'score': 0.5457841753959656}",LABEL_1
4413,4413,Kenya: Sudanese Businessman Hosts Party for Wi...,2015-02-27,John Agou is a Southern Sudanese businessman w...,The Star (Nairobi),"{'label': 'LABEL_1', 'score': 0.529257595539093}",LABEL_1
5149,5149,South Sudan: Former South Sudan Child Soldiers...,2014-07-29,"Wau — South Sudan's former child soldiers, kno...",Sudan Tribune,"{'label': 'LABEL_1', 'score': 0.5066477060317993}",LABEL_1
5886,5886,South Sudan: Young Volunteers Help Rebuild Dar...,2014-05-18,"The situation in Darfur, as in many other conf...",United Nations-African Union Mission in Darfur...,"{'label': 'LABEL_1', 'score': 0.5255243182182312}",LABEL_1
9054,9054,"South Sudan: Lakes State - One Killed, Five Wo...",2013-04-08,Juba — At least a person was killed and five o...,Sudan Tribune,"{'label': 'LABEL_1', 'score': 0.5192702412605286}",LABEL_1
10994,10994,"Sudan: Act Responsibly to Avoid War, AU Tells ...",2012-04-18,(AWEIL) The African Union (AU) on Wednesday ur...,Sudan Tribune,"{'label': 'LABEL_1', 'score': 0.5124022364616394}",LABEL_1
11268,11268,South Sudan: Unity State Opposition Critical o...,2012-08-27,Bentiu — A meeting by South Sudan's ruling Sud...,Sudan Tribune,"{'label': 'LABEL_1', 'score': 0.5335545539855957}",LABEL_1
12136,12136,South Sudan: Rebel Leader Arrives in Juba,2012-03-07,Bor — The leader for one of South Sudan's rebe...,Sudan Tribune,"{'label': 'LABEL_1', 'score': 0.51186603307724}",LABEL_1
15711,15711,Ethiopia: Ethio South African Investment Forum...,2014-06-04,A three day Ethio-South Africa Investment and ...,Ethiopian Radio and Television Agency (Addis A...,"{'label': 'LABEL_1', 'score': 0.5206533670425415}",LABEL_1
17707,17707,South Sudan: Govt Relocates Its Capital From J...,2011-09-03,Juba — The newly born Republic of South Sudan ...,Sudan Tribune,"{'label': 'LABEL_1', 'score': 0.5173115730285645}",LABEL_1


In [18]:
import pandas as pd
import ast

df_summary = pd.read_csv("data/articles_summary_cleaned.csv")
df_sentiment = pd.read_csv("data/all_africa_southsudan_sentiment_default.csv", index_col=0)
df_sentiment['sentiment'] = df_sentiment['sentiment'].apply(ast.literal_eval)


def get_sentiment(row):
    if row['sentiment'][0]['score'] > 0.5:
        return row['sentiment'][0]['label']
    return 'NEUTRAL'

df_sentiment['label'] = df_sentiment.apply(get_sentiment, axis=1)



In [25]:
df_summary.head()

,summary,date,location_article,lat,lng,sentiment
0,The article discusses the passing of the new C...,2011-07-07,Juba,4.859363,31.571250,0.991426
1,The article discusses the military actions tak...,2011-07-03,Abyei,9.838551,28.486396,-0.997186
2,The article discusses the signing of a Framewo...,2011-06-30,Southern Kordofan,11.036544,30.895824,0.982812
3,The article discusses the upcoming independenc...,2011-07-04,South Sudan,6.876992,31.306979,0.995117
4,The article discusses the need for South Sudan...,2011-07-02,Juba,4.859363,31.571250,0.791571


In [27]:
df_sentiment.head()

,title,date,paragraphs,publisher,sentiment,label
0,South Sudan: A Former Child Soldier Fights to ...,2021-02-12,"Sara, who was abducted at age 12, spent two ye...",Thomson Reuters Foundation (London),"[{'label': 'NEGATIVE', 'score': 0.997965574264...",NEGATIVE
1,South Sudan: #budget2019 - Will It Lead to Mor...,2019-02-21,Legal claims against the department now amount...,Bhekisisa (Johannesburg),"[{'label': 'NEGATIVE', 'score': 0.984295010566...",NEGATIVE
2,South Sudan: Retired NBA Star Elected Basketba...,2019-11-25,Former NBA star Luol Ajou Deng has been electe...,The East African (Nairobi),"[{'label': 'POSITIVE', 'score': 0.675956308841...",POSITIVE
3,"South Sudan - Political Violence On the Rise, ...",2022-02-11,UN human rights experts have warned of increas...,UN News Service,"[{'label': 'NEGATIVE', 'score': 0.989909410476...",NEGATIVE
4,South Sudan: S. Sudan Receives First Astrazene...,2021-03-25,"Juba — South Sudan on Thursday received 132,00...",The East African (Nairobi),"[{'label': 'POSITIVE', 'score': 0.998383641242...",POSITIVE


In [ ]:
original_df = pd.read_csv("data/articles_summary_cleaned.csv", parse_dates=["date"])

In [26]:
df_combined = pd.merge(df_summary, df_sentiment, on=["date", "lat", "lng"], how="left")

KeyError: 'lat'

In [1]:
import pandas as pd

df = pd.read_csv("data/all_africa_southsudan_sentiment_default.csv")

df.head()

,Unnamed: 0,title,date,paragraphs,publisher,sentiment
0,0,South Sudan: A Former Child Soldier Fights to ...,2021-02-12,"Sara, who was abducted at age 12, spent two ye...",Thomson Reuters Foundation (London),"[{'label': 'NEGATIVE', 'score': 0.997965574264..."
1,1,South Sudan: #budget2019 - Will It Lead to Mor...,2019-02-21,Legal claims against the department now amount...,Bhekisisa (Johannesburg),"[{'label': 'NEGATIVE', 'score': 0.984295010566..."
2,2,South Sudan: Retired NBA Star Elected Basketba...,2019-11-25,Former NBA star Luol Ajou Deng has been electe...,The East African (Nairobi),"[{'label': 'POSITIVE', 'score': 0.675956308841..."
3,3,"South Sudan - Political Violence On the Rise, ...",2022-02-11,UN human rights experts have warned of increas...,UN News Service,"[{'label': 'NEGATIVE', 'score': 0.989909410476..."
4,4,South Sudan: S. Sudan Receives First Astrazene...,2021-03-25,"Juba — South Sudan on Thursday received 132,00...",The East African (Nairobi),"[{'label': 'POSITIVE', 'score': 0.998383641242..."


In [3]:
import pandas as pd
import difflib
import ast
def get_sentiment(row):
    if row['sentiment'][0]['score'] > 0.5:
        return row['sentiment'][0]['label']
    return 'NEUTRAL'

def get_label_from_score(row):
    if row['sentiment']> 0.5:
        return 'POSITIVE'
    elif row['sentiment'] < -0.5:
        return 'NEGATIVE'
    return 'NEUTRAL'


    

df_full = pd.read_csv("data/full_dataset.csv")
df_all_africa = pd.read_csv("data/all_africa_southsudan_sentiment_default.csv", index_col=0)
df_all_africa['sentiment'] = df_all_africa['sentiment'].apply(ast.literal_eval)
df_all_africa['label'] = df_all_africa.apply(get_sentiment, axis=1)
df_full.drop_duplicates(["summary"], inplace=True)
df_summary = pd.read_csv("data/articles_summary_cleaned.csv")
df_summary["sentiment"] = df_summary.apply(get_label_from_score, axis=1)
df_full = pd.merge(df_full, df_summary[["summary", "sentiment"]], on="summary", how="left")
df_full.rename(columns={"sentiment": "sentiment_summary"}, inplace=True)




In [16]:
df_full.head()

,summary,date,name,lat,lng,id,paragraphs,publisher,tags,sentiment_summary
0,The article discusses the passing of the new C...,2011-07-07,Juba,4.859363,31.571250,0,Juba — THE parliament of South Sudan has passe...,New Vision (Kampala),"['South Sudan', 'East Africa', 'Governance', '...",POSITIVE
1,The article discusses the military actions tak...,2011-07-03,Abyei,9.838551,28.486396,1,What is the logic of Khartoum's military actio...,South Sudan News Agency,"['South Sudan', 'East Africa', 'Business', 'Co...",NEGATIVE
2,The article discusses the signing of a Framewo...,2011-06-30,Southern Kordofan,11.036544,30.895824,2,Secretary-General Ban Ki-moon today welcomed t...,UN News Service,"['Sudan', 'East Africa', 'Governance', 'Extern...",POSITIVE
3,The article discusses the upcoming independenc...,2011-07-04,South Sudan,6.876992,31.306979,3,AS South Sudan readies itself to become the wo...,The Moment (London),"['Sudan', 'East Africa', 'Africa', 'South Sudan']",POSITIVE
4,The article discusses the need for South Sudan...,2011-07-02,Juba,4.859363,31.571250,4,Juba — The Vice President of the soon-to-be in...,Sudan Tribune,"['South Sudan', 'East Africa', 'Business', 'Ex...",POSITIVE


In [1]:
import tqdm

tqdm.tqdm.pandas()

df_all_africa["paragraph_len"] = df_all_africa["paragraphs"].apply(len)
lowbound = 20
highbound = 65

df_all_africa["paragraph_section"] = df_all_africa["paragraphs"].apply(lambda x: x[lowbound:highbound])
paragraphs = df_full["paragraphs"].unique().tolist()
paragraphs.sort()
def find_corresponding_paragraph(paragraphs, paragraph_section):
    res = list(filter(lambda x: paragraph_section in x, paragraphs))
    if len(res) > 0:
        paragraphs.remove(res[0])
        return res[0]
    return None
df_all_africa["paragraph"] = df_all_africa.progress_apply(lambda x: find_corresponding_paragraph(paragraphs, x["paragraph_section"]), axis=1)
df_all_africa.head()

NameError: name 'df_all_africa' is not defined

In [37]:
df_full["paragraphs"]

0        Juba — THE parliament of South Sudan has passe...
1        What is the logic of Khartoum's military actio...
2        Secretary-General Ban Ki-moon today welcomed t...
3        AS South Sudan readies itself to become the wo...
4        Juba — The Vice President of the soon-to-be in...
                               ...                        
19880    When Grace* arrived at the Maiwut Primary Heal...
19881    To date, out of 86 major and essential hospita...
19882    Addis Ababa — Prime Minister Abiy Ahmed has ca...
19883    One person was killed on Sunday and four other...
19884    North Darfur State Governor Nimir Mohamed and ...
Name: paragraphs, Length: 19885, dtype: object

In [40]:
df_full = pd.merge(df_full, df_all_africa[["paragraph", "label"]], left_on="paragraphs",  right_on="paragraph", how="left")


In [47]:


df_full.paragraph.notna().sum(), df_full.paragraph.isna().sum()
df_full.dropna(subset=["paragraph"], inplace=True)

df_full.rename(columns={"label": "sentiment_paragraph"}, inplace=True)
df_full.drop(["paragraph", "paragraphs_len"], axis=1, inplace=True)
df_full.to_csv("data/full_dataset_with_sentiment.csv", index=True)

In [23]:
import pandas as pd
df_full = pd.read_csv("data/full_dataset_with_sentiment.csv", index_col=0)
df_full

,summary,date,name,lat,lng,id,paragraphs,publisher,tags,sentiment_summary,sentiment_paragraph
0,The article discusses the passing of the new C...,2011-07-07,Juba,4.859363,31.571250,0,Juba — THE parliament of South Sudan has passe...,New Vision (Kampala),"['South Sudan', 'East Africa', 'Governance', '...",POSITIVE,NEGATIVE
1,The article discusses the military actions tak...,2011-07-03,Abyei,9.838551,28.486396,1,What is the logic of Khartoum's military actio...,South Sudan News Agency,"['South Sudan', 'East Africa', 'Business', 'Co...",NEGATIVE,NEGATIVE
2,The article discusses the signing of a Framewo...,2011-06-30,Southern Kordofan,11.036544,30.895824,2,Secretary-General Ban Ki-moon today welcomed t...,UN News Service,"['Sudan', 'East Africa', 'Governance', 'Extern...",POSITIVE,POSITIVE
3,The article discusses the upcoming independenc...,2011-07-04,South Sudan,6.876992,31.306979,3,AS South Sudan readies itself to become the wo...,The Moment (London),"['Sudan', 'East Africa', 'Africa', 'South Sudan']",POSITIVE,NEGATIVE
5,The article discusses the United States' respo...,2011-07-06,Addis Ababa,8.980603,38.757761,5,U.S. DEPARTMENT OF STATE Office of the Spokesp...,"United States Department of State (Washington,...","['Sudan', 'East Africa', 'Governance', 'Extern...",POSITIVE,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...
19872,The article discusses the call made by Kenyan ...,2022-12-25,Upper Nile,9.889420,32.718137,20870,Nairobi — President William Ruto has called on...,Capital FM (Nairobi),"['Kenya', 'East Africa', 'Conflict', 'South Su...",NEGATIVE,NEGATIVE
19873,The article discusses South Sudan's first Inte...,2023-02-20,Radisson in Juba,4.859363,31.571250,20871,"South Sudan, Africa's youngest nation, looked ...",Liberian Observer (Monrovia),"['Women', 'East Africa', 'West Africa', 'Gover...",POSITIVE,POSITIVE
19874,The article discusses Uganda's military leader...,2023-03-07,"Mbuya, Kampala",0.325589,32.630572,20872,Uganda 's military adventurism is set to be te...,Nile Post News (Kampala),"['Uganda', 'East Africa', 'Conflict', 'Arms an...",NEGATIVE,NEGATIVE
19875,The article discusses an upcoming ecumenical v...,2023-01-27,Juba,4.859363,31.571250,20873,"Juba — As part of an ecumenical visit, Pope Fr...",Fides News Agency (Vatican),"['South Sudan', 'East Africa', 'Religion']",POSITIVE,POSITIVE


In [56]:
df_full.loc[2, "paragraphs"]

'Secretary-General Ban Ki-moon today welcomed the signing of an agreement between the governments of Sudan and Southern Sudan on border security and a joint political and security arrangement, urging both parties to conclude and implement a cessation of hostilities pact as well. The Framework Agreement for Southern Kordofan and Blue Nile states, which lie on the border between the north and the south, was signed yesterday in the Ethiopian capital, Addis Ababa. Troops from the north have in recent weeks clashed with armed groups aligned with the south in South Kordofan, sparking a humanitarian crisis as thousands of people fled from the fighting in and around the town of Kadugli. Southern Sudan is set to become an independent State on July 9 after its residents voted in favour of secession in a referendum held in January. Mr. Ban said that unless both the northern and southern government reached an agreement on the cessation of hostilities, the United Nations peacekeeping mission in Sud

In [54]:
df_full.loc[1]

summary                The article discusses the military actions tak...
date                                                          2011-07-03
name                                                               Abyei
lat                                                             9.838551
lng                                                            28.486396
id                                                                     1
paragraphs             What is the logic of Khartoum's military actio...
publisher                                        South Sudan News Agency
tags                   ['South Sudan', 'East Africa', 'Business', 'Co...
sentiment_summary                                               NEGATIVE
sentiment_paragraph                                             NEGATIVE
Name: 1, dtype: object

In [64]:

# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer, RobertaForSequenceClassification

pipe = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")



tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", problem_type="sentiment-analysis")


df_full["sentiment_roberta"] = df_full["paragraphs"].progress_apply(lambda x: pipe(x)[0]["label"])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassifi

RuntimeError: The expanded size of the tensor (615) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 615].  Tensor sizes: [1, 514]

In [79]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ImportError: 
TFAutoModelForSequenceClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


In [78]:

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

def get_sentiment_roberta(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')

    output = model(**encoded_input)
    print(output)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return np.argmax(scores)


df_full["sentiment_roberta"] = df_full["paragraphs"].progress_apply(get_sentiment_roberta)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 1/17018 [00:00<01:08, 249.75it/s]


RuntimeError: The expanded size of the tensor (615) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 615].  Tensor sizes: [1, 514]

In [27]:
from transformers import pipeline, AutoTokenizer
from flair.embeddings import TransformerWordEmbeddings
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_path, padding='max_length', max_length=512, truncation=True)
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=tokenizer, max_length=512, truncation=True)

sentiment_task(df_full.loc[1, "paragraphs"])

[{'label': 'negative', 'score': 0.8014981746673584}]

In [34]:

for i in range(500):
    print(sentiment_task(df_full.loc[df_full.index[i], "paragraphs"]))

[{'label': 'neutral', 'score': 0.6936390995979309}]
[{'label': 'negative', 'score': 0.8014981746673584}]
[{'label': 'neutral', 'score': 0.601162314414978}]
[{'label': 'neutral', 'score': 0.7806603312492371}]
[{'label': 'neutral', 'score': 0.621727466583252}]
[{'label': 'neutral', 'score': 0.682358980178833}]
[{'label': 'neutral', 'score': 0.6789297461509705}]
[{'label': 'positive', 'score': 0.7867504358291626}]
[{'label': 'neutral', 'score': 0.8907796144485474}]
[{'label': 'neutral', 'score': 0.8308969736099243}]
[{'label': 'neutral', 'score': 0.7046489715576172}]
[{'label': 'positive', 'score': 0.5893830060958862}]
[{'label': 'neutral', 'score': 0.7150486707687378}]
[{'label': 'neutral', 'score': 0.6906048059463501}]
[{'label': 'neutral', 'score': 0.7813199162483215}]
[{'label': 'neutral', 'score': 0.6996539235115051}]
[{'label': 'neutral', 'score': 0.6633765697479248}]
[{'label': 'negative', 'score': 0.4973313808441162}]
[{'label': 'negative', 'score': 0.5027934312820435}]
[{'label':

KeyboardInterrupt: 

In [21]:
def get_sentiment_roberta(row):
    sent = sentiment_task(row)
    if sent[0]["label"] == "positive":
        if sent[0]["score"] > 0.5:
            return "POSITIVE"
    
    elif sent[0]["label"] == "negative":
        if sent[0]["score"] > 0.5:
            return "NEGATIVE"
    
    return "neutral"

from tqdm import tqdm
tqdm.pandas()
df_full["sentiment_roberta"] = df_full["paragraphs"].progress_apply(get_sentiment_roberta)
    
    

  0%|          | 71/17018 [01:01<4:04:53,  1.15it/s]


KeyboardInterrupt: 

In [22]:
import multiprocessing

from multiprocessing import Process
import numpy as np


def get_sentiment_roberta_split(df_section, start, end):
    df_section["sentiment_roberta"] = df_section["paragraphs"].progress_apply(get_sentiment_roberta)
    df.to_csv(f"data/full_dataset_with_sentiment_{start}_{end}.csv", index=True)

if __name__ == "__main__":  # confirms that the code is under main function
    # create a list of indexes to split the dataframe into 10 parts
    df_split = np.array_split(df_full, 10)
    # create processes equal to the number of parts
    processes = []
    for i in range(10):
        p = multiprocessing.Process(target=get_sentiment_roberta_split, args=(df_split[i], df_split[i].index[0], df_split[i].index[-1]))
        processes.append(p)
        p.start()
    # complete the processes
    for proc in processes:
        proc.join()


12

In [94]:

def print_func(continent='Asia'):
    print('The name of continent is : ', continent)

if __name__ == "__main__":  # confirms that the code is under main function
    names = ['America', 'Europe', 'Africa']
    procs = []
    proc = Process(target=print_func)  # instantiating without any argument
    procs.append(proc)
    proc.start()

    # instantiating process with arguments
    for name in names:
        # print(name)
        proc = Process(target=print_func, args=(name,))
        procs.append(proc)
        proc.start()

    # complete the processes
    for proc in procs:
        proc.join()

^C


     ---------------------------------------- 0.0/373.1 kB ? eta -:--:--
     ------------------------------------  368.6/373.1 kB 11.6 MB/s eta 0:00:01
     -------------------------------------- 373.1/373.1 kB 5.8 MB/s eta 0:00:00
  Obtaining dependency information for gensim>=3.8.0 from https://files.pythonhosted.org/packages/d3/68/373da90a8b241e2603707c7aa4c8f47829a72729c6e9497f2bc604fa6a6a/gensim-4.3.2-cp39-cp39-win_amd64.whl.metadata
  Using cached gensim-4.3.2-cp39-cp39-win_amd64.whl.metadata (8.5 kB)
     ---------------------------------------- 0.0/788.5 kB ? eta -:--:--
     ----------------- ------------------- 368.6/788.5 kB 11.2 MB/s eta 0:00:01
     ------------------------------------  788.5/788.5 kB 10.0 MB/s eta 0:00:01
     -------------------------------------- 788.5/788.5 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements t

ERROR: Could not install packages due to an OSError: [WinError 5] Toegang geweigerd: 'C:\\Users\\barta\\Desktop\\Uni\\Data Challange 3\\JBG060-DC3-Group-25\\dc3env\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Check the permissions.



In [24]:
import pandas as pd

df_full = pd.read_csv("data/full_dataset_with_sentiment_combined.csv", index_col=0)

In [26]:
df_full.sentiment_roberta.value_counts()

NEUTRAL    17018
Name: sentiment_roberta, dtype: int64

In [37]:
import pandas as pd

df_full = pd.read_csv("data/full_dataset_with_sentiment_combined2.csv", index_col=0)

df_full[df_full["sentiment_roberta"] == df_full["sentiment_roberta_sum"]].shape[0] / df_full.shape[0]

0.7468419506462984

In [8]:
df = pd.read_csv("data/full_dataset_with_sentiment_combined2.csv", index_col=0)
df_sum = pd.read_csv("data/articles_summary_cleaned.csv")[["location_article", "summary"]]
combined = pd.merge(df, df_sum, on=["summary"], how="left")

combined.to_csv("data/full_dataset_with_sentiment_combined2.csv", index=True)

In [12]:
df = pd.read_csv("data/food_crises_cleaned.csv")

In [14]:
df[df["ipc"] == 5]

,country,district_code,district,centx,centy,year_month,year,month,ipc,ha,...,et_anom,count_violence,sum_fatalities,food_price_idx,area,cropland_pct,pop,ruggedness_mean,pasture_pct,date
7547,South Sudan,121,Leer,30.22817,8.174243,2017_02,2017,2,5.0,0.0,...,0.201752,3,32,20.40088,1637.891,41.07024,97790.0,36786.08,86.84211,2017-02-01
